In [1]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline


## Building the Value Class

In [58]:
class Value: 
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda:None
        self._prev = set(_children)
        self._op = _op


    def __repr__(self) -> str:
        return f"Value(data = {self.data})"
    
    def __add__(self, other): 
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward(): 
            self.grad += out.grad
            other.grad += out.grad

        out._backward = _backward

        return out
    
    def __mul__(self, other): 
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward(): 
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out
    
    def __sub__(self, other): 
        other = other if isinstance(other, Value) else Value(other)
        return self + (-other)
    
    def __neg__(self): 
        return self * -1
    
    def __pow__(self, other): 
        assert isinstance(other, (int, float)), "only supporting int/float powers"
        out = Value(self.data ** other, (self, ), f'**{other}')

        def _backward(): 
            self.grad += other * (self.data ** (other -1)) * out.grad
        out._backward = _backward

        return out
    
    def tanh(self): 
        x = self.data 
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward(): 
            self.grad += (1 - t**2) * out.grad

        out._backward = _backward

        return out
    
    def backward(self): 

        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        # Go one variable at a time and apply the chain rule
        self.grad = 1.0
        for v in reversed(topo):
            v._backward()


In [62]:
a = Value(2.0)
b = Value(3.0)

c = b + a
print(f"c.data: {c.data}")  # Should be 5.0

d = c ** 2
print(f"d.data: {d.data}")  # Should be 25.0

e = d * 10
print(f"e.data: {e.data}")  # Should be 250.0

e.backward()

print("\nGradients:")
print(f"e.grad: {e.grad}")
print(f"d.grad: {d.grad}")
print(f"c.grad: {c.grad}")
print(f"b.grad: {b.grad}")
print(f"a.grad: {a.grad}")

c.data: 5.0
d.data: 25.0
e.data: 250.0

Gradients:
e.grad: 1.0
d.grad: 10.0
c.grad: 0.0
b.grad: 0.0
a.grad: 0.0
